In [29]:
import pandas as pd
from src.parsers import parse_all_comdirect
from src.dashboard_utility import get_monthly_data
import dateparser

In [2]:
path = "finance_data/comdirect/"

df = parse_all_comdirect(path)

/Users/hannesmoehring/Documents/Projekte/dev/finance_overview/src/parsers.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


In [3]:
monthly_data = get_monthly_data(df)
monthly_data

,month,type,amount
0,2025-01,Income,3104.12
1,2025-01,Spending,2649.64
2,2025-02,Income,799.12
3,2025-02,Spending,962.68
4,2025-04,Income,830.12
5,2025-04,Spending,112.14
6,2025-05,Income,1220.12
7,2025-05,Spending,1230.09
8,2025-06,Income,2334.12
9,2025-06,Spending,2416.04


In [25]:
from pypdf import PdfReader

path = "finance_data/traderepublic/Account_statement.pdf"
reader = PdfReader(path)

print("Number of pages:", len(reader.pages))

all_text = []
for page in reader.pages:
    text = page.extract_text()
    all_text.append(text)


full_text = "\n\n\n".join(all_text)
lines = full_text.splitlines()
filtered_lines = []
i = 0
while i < len(lines):
    if "Kauf" in lines[i] or "Verkauf" in lines[i]:
        new_line = lines[i] + " " + lines[i + 1]
        filtered_lines.append(new_line)
        i += 2
        continue
    if "Überweisung" in lines[i] or "Kartentransaktion" in lines[i]:
        dates = lines[i - 3] + lines[i - 2] +  lines[i - 1]
        if len(dates) > 14:
            new_line = lines[i - 1] + lines[i]
            filtered_lines.append(new_line)
            i += 1
            continue
        new_line = lines[i - 3] + lines[i - 2] +  lines[i - 1] + " " + lines[i]
        filtered_lines.append(new_line)
        i += 1
        continue
    else:
        i += 1

for line in filtered_lines:
    print(line)

# print(full_text)


Number of pages: 24
08 Jan. 2025 Kartentransaktion UBER *TRIP 27,96 € 3.519,49 €
16 Jan. 2025 Kartentransaktion Draxlhanglifte 39,00 € 3.481,08 €
16 Jan. 2025 Kartentransaktion Panoramarestaurant am Bra 25,00 € 3.456,08 €
17 Jan. 2025 Überweisung Outgoing transfer for Hannes Moehring 100,00 € 3.354,51 €
20 Jan. 2025 Kartentransaktion Hoflinger 10091 7,40 € 2.847,05 €
23 Jan. 2025 Überweisung Apple Pay Top up 1.300,00 € 4.044,22 €
24 Jan. 2025 Kartentransaktion I.M. Nahkauf GmbH 13,52 € 4.030,10 €
16 Feb. 2025 Kartentransaktion PAYPAL *ADIDAS AG 3031 110,00 € 4.400,80 €
17 Feb. 2025 Überweisung Outgoing transfer for Hannes Moehring 109,00 € 4.291,80 €
22 Feb. 2025 Kartentransaktion PAYPAL *JANICKB 43,70 € 4.282,14 €
24 Feb. 2025 Kartentransaktion Dominos Muenchen 71,72 € 4.210,42 €
24 Apr. 2025 Kartentransaktion PAYPAL *johannesgrymer 90,00 € 4.459,18 €
29 Apr. 2025 Kartentransaktion PAYPAL *FLUGHAFENMU 49,99 € 4.409,19 €
12 Mai 2025 Kartentransaktion PAYPAL *BLUMENTIMMG BL 34,34 € 4.37

In [64]:
traderepublic_df = pd.DataFrame(columns=["date", "process", "details", "amount"])

new_row_template = {"date": "", "process": "", "details": "", "amount": ""}
for line in filtered_lines:
    new_row = new_row_template.copy()
    line_split = line.split(" ")
    dates = line_split[:3]
    dt1 = dateparser.parse(" ".join(dates), settings={"DATE_ORDER": "DMY", "PREFER_DAY_OF_MONTH": "first"})
    assert dt1 is not None
    new_row["date"] = dt1.date()
    new_row["process"] = line_split[3]
    new_row["amount"] = line_split[-2].split("\xa0")[0].replace(".", "").replace(",", ".")
    if line_split[3] == "Überweisung":
        if line_split[4] == "Outgoing":
            new_row["details"] = " ".join(line_split[6:9])
            new_row["amount"] = "-" + new_row["amount"]
        elif line_split[4] == "Incoming":
            new_row["details"] = " ".join(line_split[6:9])    
    else:
        new_row["details"] = " ".join(line_split[4:6])
    if new_row["process"] == "Kartentransaktion" or new_row["process"] == "Kauf":
        new_row["amount"] = "-" + new_row["amount"]

    # print(new_row)
    traderepublic_df = pd.concat([traderepublic_df, pd.DataFrame([new_row])], ignore_index=True)



    

In [65]:
traderepublic_df['amount'] = pd.to_numeric(traderepublic_df['amount'])
traderepublic_df['date'] = pd.to_datetime(traderepublic_df['date'])
traderepublic_df.sort_values(by='date', inplace=True)
traderepublic_df.reset_index(drop=True, inplace=True)

In [66]:
traderepublic_df

,date,process,details,amount
0,2025-01-08,Kartentransaktion,UBER *TRIP,-27.96
1,2025-01-16,Kartentransaktion,"Draxlhanglifte 39,00 €",-39.00
2,2025-01-16,Kartentransaktion,Panoramarestaurant am,-25.00
3,2025-01-17,Überweisung,for Hannes Moehring,-100.00
4,2025-01-20,Kartentransaktion,Hoflinger 10091,-7.40
...,...,...,...,...
144,2025-10-28,Kauf,BlackRock ICS,-1.00
145,2025-10-29,Kartentransaktion,AMZN Mktp,-13.99
146,2025-10-30,Verkauf,BlackRock ICS,1.00
147,2025-10-31,Kartentransaktion,AMZN Mktp,-25.99
